In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

# Tus importaciones originales se mantienen
import polars as pl
import numpy as np
from datetime import date
from tqdm.notebook import tqdm
import itertools
from pathlib import Path
import random
import copy

In [ ]:
def set_seed(seed: int):
    """Fija la semilla para todas las librerías relevantes para la reproducibilidad."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
        
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
def add_features(df: pl.DataFrame, columnas: list) -> pl.DataFrame:        
    """... (código original sin cambios) ..."""
    # Lags para features de mercado
    for column in columnas:
        if column in ['date', 'price']:
            continue
        df = df.with_columns(pl.col(column).shift(1).alias(f'{column}_lag_1'))

    # **CALCULAR _change_1d
    for column in columnas:
        if column in ['date', 'total_volume']:
            continue
        df = df.with_columns(((pl.col(column) - pl.col(column).shift(1)) / pl.col(column).shift(1)).alias(f"{column}_change_1d")  )     

    # Ratios intermercado
    for column in columnas:
        if column in ['date', 'price', 'total_volume']:
            continue
        df = df.with_columns((pl.col("price") / pl.col(column)).alias(f"btc_{column}_ratio"))

    df = df.with_columns([
        pl.when(pl.col("price_change_1d") > 0)
          .then(1)
          .when(pl.col("price_change_1d") < 0)
          .then(-1)
          .otherwise(0)
          .alias("price_direction_1d"),
    ])
    # Features temporales básicas
    df = df.with_columns([
        pl.col("date").dt.year().alias("year"),
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.day().alias("day"),
        pl.col("date").dt.weekday().alias("weekday"),
    ])
    # Lags de precio Variados
    lags = [15, 30, 45, 60, 75, 90]
    for lag in lags:
        df = df.with_columns([
            # Cambios en lags propuestos
            pl.col("price").shift(lag).alias(f"btc_lag_{lag}"),
        ])
    # Rolling statistics de precio
    windows = [3, 7, 14, 21, 30]
    for window in windows:
        df = df.with_columns([
            pl.col("price").rolling_std(window).alias(f"btc_std_{window}"),
        ])
    windows = [30, 60, 90]
    for window in windows:
        df = df.with_columns([
            # Medias de los Instrumentos en Ciertas Ventanas
            pl.col("price").rolling_mean(window).alias(f"price_ma_{window}"),
            # Momentum adicional basado en precio (no confundir con price_change_1d)
            ((pl.col("price") / pl.col(f"btc_lag_{window}")) - 1).alias(f"btc_momentum_{window}d"),
        ])
    # **TARGET: precio de mañana (SIEMPRE AL FINAL)**
    df = df.with_columns(pl.col("price").shift(-1).alias("price_tomorrow"))
    df = df.with_columns(
        pl.when((pl.col('price_tomorrow') - pl.col('price')) > 0)
        .then(1)
        .when((pl.col('price_tomorrow') - pl.col('price')) < 0)
        .then(-1)
        .otherwise(0)
        .alias('target_direction')
    )
    # Eliminar NaNs (ÚLTIMO PASO)
    max_offset = max(max(lags, default=0), max(windows, default=0), 1)
    return (df.slice(max_offset, df.shape[0] - max_offset)).drop_nulls()

In [4]:
def temporal_split(df: pl.DataFrame, target: str, test_size: float = 0.4, fecha_corte: date = date(2024, 12, 31)):
    # ... (Esta función también se mantiene casi igual, solo se añade el escalado de datos)
    """... (código original con la adición del scaler) ..."""
    df_trainval = df.filter(pl.col("date") <= fecha_corte)
    df_future = df.filter(pl.col("date") > fecha_corte)
    df_trainval = df_trainval.sort("date")
    df_future = df_future.sort("date")
    split_idx = int(len(df_trainval) * (1 - test_size))
    df_train = df_trainval.slice(0, split_idx)
    df_test = df_trainval.slice(split_idx, len(df_trainval) - split_idx)
    feature_cols = [col for col in df_train.columns if col not in ["date", target, 'price_tomorrow']]

    # --- ¡NUEVO Y MUY IMPORTANTE PARA REDES NEURONALES! ---
    scaler = StandardScaler()
    X_train = scaler.fit_transform(df_train.select(feature_cols).to_numpy())
    X_test = scaler.transform(df_test.select(feature_cols).to_numpy())
    X_test_future = scaler.transform(df_future.select(feature_cols).to_numpy())
    # --- FIN DE LA SECCIÓN NUEVA ---

    y_train = df_train.select(target).to_numpy().flatten()
    y_test = df_test.select(target).to_numpy().flatten()
    y_test_future = df_future.select(target).to_numpy().flatten()
    return df_trainval, df_future, X_train, y_train, X_test, y_test, df_test, df_train, X_test_future, y_test_future, scaler

## Definición del Modelo Pytorch
#### `MLP (Multi-Layer Perceptron)`
- **Capa Lineal (nn.Linear):**
    - Realiza una transformación lineal (y=Wx+b). Es la capa que "aprende" las relaciones entre las features.
- **Activación (nn.ReLU):** 
    - Introduce la no-linealidad. Sin esto, apilar capas lineales sería matemáticamente igual a tener una sola capa lineal, y el modelo no podría aprender patrones complejos. ReLU simplemente convierte todos los valores negativos en cero.
- **Regularización (nn.Dropout):** 
    - Como vimos, combate el sobreajuste.
- **Capa de Salida:** 
    - Es una capa lineal con una sola neurona para producir un único valor de salida (nuestra predicción de target_direction).

In [6]:
class BitcoinDirectionNet(nn.Module):
    """Red neuronal optimizada para predecir dirección del precio de Bitcoin."""
    
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout_prob=0.3):
        super(BitcoinDirectionNet, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        # Construcción dinámica de capas
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),  # BatchNorm para mejor estabilidad
                nn.LeakyReLU(0.1),  # LeakyReLU para evitar neuronas muertas
                nn.Dropout(dropout_prob)
            ])
            prev_dim = hidden_dim
        
        # Capa final de clasificación
        layers.append(nn.Linear(prev_dim, 1))
        layers.append(nn.Sigmoid())  # Sigmoid para probabilidades [0,1]
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

In [7]:
def evaluar_modelo_pytorch(
    modelo: nn.Module,
    data_loader: DataLoader,
    df_test: pl.DataFrame,
    device: torch.device,
    transaction_cost_pct: float = 0.001,
    threshold: float = 0.5  # Umbral para clasificación
) -> dict:
    """Función de evaluación adaptada para PyTorch."""
    modelo.eval()
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for features, labels in data_loader:
            features = features.to(device)
            outputs = modelo(features)
            probs = outputs.cpu().numpy().flatten()
            preds = (probs > threshold).astype(int) * 2 - 1  # Convertir a -1/1
            all_preds.extend(preds)
            all_probs.extend(probs)

    preds = np.array(all_preds)

    df_test = df_test.sort('date')
    prices_hoy = df_test['price'].to_numpy()
    prices_tomorrow = df_test['price_tomorrow'].to_numpy()
    y_test = df_test['target_direction'].to_numpy()  # Obtenemos y_test del df

    # El resto de la lógica de cálculo de métricas es idéntica
    aciertos_direccion = 0
    retornos = []
    retornos_porcentuales = []
    tasas_libre_riesgos = []
    for i in range(len(preds)):
        retorno = abs(prices_tomorrow[i] - prices_hoy[i])
        tasa_libre = prices_hoy[i] * transaction_cost_pct
        tasas_libre_riesgos.append(tasa_libre)
        if (preds[i] > 0.5 and y_test[i] == 1) or (preds[i] < 0.5 and y_test[i] == -1):
            aciertos_direccion += 1
            retornos.append(retorno)
            retornos_porcentuales.append((retorno - tasa_libre) / prices_hoy[i])
        else:
            retornos.append(-retorno)
            retornos_porcentuales.append(-(retorno + tasa_libre) / prices_hoy[i])

    exce_retorno = np.array(retornos) - np.array(tasas_libre_riesgos)
    sharpe_ratio = np.mean(exce_retorno) / np.std(exce_retorno) if np.std(exce_retorno) != 0 else 0
    directional_accuracy = aciertos_direccion / len(preds)
    n_evaluated_days = len(preds)
    retornos_porcentuales_np = np.array(retornos_porcentuales)
    cumulative_return = np.prod(1 + retornos_porcentuales_np) - 1
    cagr = (1 + cumulative_return)**(365 / n_evaluated_days) - 1

    return {
        "directional_accuracy": directional_accuracy,
        "sharpe_ratio": sharpe_ratio,
        "cumulative_return": cumulative_return,
        "Compound_Annual_Growth_Rate": cagr,
        "n_evaluated_days": n_evaluated_days,
    }

In [8]:
def entrena_evalua_pytorch(
    hidden_dim: int,
    learning_rate: float,
    dropout: float,
    epochs: int,
    batch_size: int,
    train_loader: DataLoader,
    test_loader: DataLoader,
    future_loader: DataLoader,
    df_test: pl.DataFrame,
    df_future: pl.DataFrame,
    device: torch.device,
    iteracion: int = 0,
    return_model: bool=False
) -> dict:

    input_dim = next(iter(train_loader))[0].shape[1]

    # Definir la arquitectura exacta
    hidden_dims = [hidden_dim, hidden_dim//2, hidden_dim//4]

    # 1. Instanciar modelo, función de pérdida y optimizador
    # model = BitcoinDirectionNet(input_dim, [hidden_dim, hidden_dim//2, hidden_dim//4], dropout).to(device)
    model = BitcoinDirectionNet(input_dim, hidden_dims, dropout).to(device)
    
    # Cambiar a pérdida de clasificación binaria
    criterion = nn.BCELoss()  # Binary Cross Entropy para clasificación
    
    # Optimizador con weight decay (regularización L2)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    # Scheduler para ajustar learning rate
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

    # --- INICIALIZACIÓN DE EARLY STOPPING ---
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    # -----------------------------------------
    data_epoch_loss = []
    for epoch in range(epochs):

        # Bucle de entrenamiento
        model.train()  # Poner el modelo en modo de entrenamiento
        epoch_loss = 0
        batch_count = 0
        running_train_loss = 0.0 
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            # Convertir labels a formato binario [0,1]
            # Asumiendo que labels son -1/1, convertimos a 0/1
            binary_labels = (labels > 0).float().unsqueeze(1)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, binary_labels)

            # Backward pass y optimización
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping para estabilidad
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
            batch_count += 1

            running_train_loss += loss.item()
        # ------------------------  ELIMINAR ESTE BLOQUE SI VA EL DE ABAJO    
        avg_epoch_loss = epoch_loss / batch_count
        scheduler.step(avg_epoch_loss)


        avg_train_loss = running_train_loss / len(train_loader)
        diccionario_epoch_loss = {}
        diccionario_epoch_loss['epoch'] = epoch
        diccionario_epoch_loss['train_loss'] = round(avg_train_loss, 4)
        diccionario_epoch_loss['lr'] = optimizer.param_groups[0]['lr']
        data_epoch_loss.append(diccionario_epoch_loss)
        # -----------------------------------------------------------------

    model.eval()
    if return_model:
        return model, optimizer
    
    # 3. Evaluación
    metricas_test = evaluar_modelo_pytorch(model, test_loader, df_test, device)
    metricas_future = evaluar_modelo_pytorch(model, future_loader, df_future, device)

    params = {
        'input_dim': input_dim,
        'hidden_dims': hidden_dims,
        'learning_rate': learning_rate,
        'dropout': dropout, 
        'epochs': epochs, 
        'batch_size': batch_size,
        'iteracion': iteracion
    }

    epochs_loss = {
        'data_epochs_loss': data_epoch_loss
    }

    return metricas_test | params, metricas_future | params, model

***
***
## Empieza el Flujo

In [9]:
set_seed(42)
df = pl.read_parquet("db/db.parquet")
target = 'target_direction'
# ['date', 'price', 'total_volume', 'market_cap', 'price_gold', 'stock_index_dowjones', 'stock_index_sp500', 'rate_US10Y', 'stock_index_ni225']
columns = ['date', 'price', 'total_volume', 'stock_index_sp500', 'price_gold', 'rate_US10Y', 'stock_index_ni225']
df = df.select(columns)
df = df.pipe(add_features, columns)

df_trainval, df_future, X_train, y_train, X_test, y_test, df_test, df_train, X_test_future, y_test_future, scaler_global = df.pipe(temporal_split, target)

### Determinar Dispositivo  (GPU si está disponible)


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


#### `hidden_dims (Tamaño de las Capas Ocultas)`
- **Teoría:**
    - Es el "poder cerebral" de tu red. Define cuántas neuronas hay en cada capa oculta. Más neuronas significan que la red tiene más capacidad para aprender patrones complejos.
- **Recomendaciones:**
    - No hay una regla de oro, pero para datos tabulares, empezar con valores entre 32 y 256 es común.
    - Una arquitectura popular es la de "embudo", donde cada capa sucesiva es más pequeña (ej., [256, 128, 64]).
    - El tamaño de la capa de entrada (el número de features) puede ser una buena guía para el tamaño de la primera capa oculta.
- **Riesgos:**
    - Valores muy altos (ej., > 512): Alto riesgo de sobreajuste. La red tiene tanta capacidad que puede memorizar cada dato de entrenamiento, incluido el ruido.
    - Valores muy bajos (ej., < 32): Riesgo de subajuste (underfitting). La red no tiene suficiente "poder" para capturar la complejidad de los datos.

#### `learning_rates (Tasa de Aprendizaje)`
- **Teoría:** 
    - Es el tamaño del paso que da el optimizador al ajustar los pesos del modelo. Es quizás el hiperparámetro más importante.
    - Imagina que estás bajando una montaña a ciegas. La tasa de aprendizaje es el tamaño de tus pasos.
- **Recomendaciones:**
    - Es un parámetro muy sensible. Los valores más comunes se mueven en un rango logarítmico: 0.01, 0.005, 0.001, 0.0005, 0.0001.
    - Empezar con 0.001 es casi siempre una apuesta segura.
- **Riesgos:**
    - Valor muy alto (ej., > 0.01): Divergencia. Los pasos son tan grandes que el modelo "salta" por encima de la solución óptima una y otra vez, y la pérdida (el error) puede explotar en lugar de disminuir.
    - Valor muy bajo (ej., < 0.0001): Convergencia muy lenta. El entrenamiento tomará una eternidad y el modelo podría quedarse atascado en una mala solución (mínimo local) porque sus pasos son demasiado tímidos para salir de ahí.

#### `dropouts (Tasa de Abandono)`
- **Teoría:**
    - Es una técnica de regularización fundamental para combatir el sobreajuste. En cada paso de entrenamiento, "apaga" aleatoriamente un porcentaje de neuronas. Esto fuerza a la red a aprender de forma más robusta, sin depender excesivamente de unas pocas neuronas.
- **Recomendaciones:**
    - Valores comunes están entre 0.1 y 0.5.
    - Un valor de 0.5 es agresivo pero muy efectivo. Un valor de 0.2 o 0.3 es un punto de partida más conservador.
- **Riesgos:**
    - Valor muy alto (ej., > 0.6): Riesgo de subajuste. Estás apagando tantas neuronas que la red no tiene suficiente capacidad para aprender adecuadamente durante el entrenamiento.
    - Valor muy bajo (ej., < 0.1): El efecto de regularización es mínimo, por lo que el riesgo de sobreajuste sigue siendo alto.

#### `epochs_list (Número de Épocas)`
- **Teoría:** 
    - Una época es una pasada completa del modelo por todo el conjunto de datos de entrenamiento. Es la cantidad de veces que el modelo "estudia" los datos.
- **Recomendaciones:**
    - Esto depende mucho del tamaño del dataset y la complejidad del problema. Puede variar desde 20 hasta 200+.
- **Práctica recomendada:**
    - En lugar de fijar un número de épocas, se usa una técnica llamada "Early Stopping" (Detención Temprana). Monitoreas el rendimiento en un set de validación y detienes el entrenamiento cuando el rendimiento en ese set deja de mejorar, evitando así el sobreajuste.
- **Riesgos:**
    - Demasiadas épocas: Causa principal de sobreajuste. El modelo aprende perfectamente los datos de entrenamiento pero pierde su capacidad de generalizar a datos nuevos.
    - Muy pocas épocas: Subajuste. El modelo no ha tenido tiempo suficiente para aprender los patrones.

#### `batch_size (Tamaño del Lote)`
- **Teoría:**
    - Es el número de muestras de datos que el modelo ve antes de actualizar sus pesos.
- **Recomendaciones:**
    - Se suelen usar potencias de 2 por optimizaciones de hardware: 32, 64, 128, 256.
    - 64 o 128 son puntos de partida excelentes.
    - Depende de la memoria de tu GPU (VRAM). Si te da un error de "out of memory", debes reducir el batch_size.
- **Riesgos:**
    - Valor muy grande: El entrenamiento es más rápido, pero puede generalizar peor, ya que los ajustes de los pesos son menos frecuentes y más "promediados".
    - Valor muy pequeño: El entrenamiento es más lento y los ajustes son más "ruidosos", lo que a veces puede ayudar a la generalización (actúa como una forma de regularización), pero puede hacer que la convergencia sea inestable.

### Creación de Tensores y DataLoaders

In [11]:
# Convertir a tensores de PyTorch
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)
X_future_t = torch.tensor(X_test_future, dtype=torch.float32)
y_future_t = torch.tensor(y_test_future, dtype=torch.float32)

# Crear Datasets
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)
future_dataset = TensorDataset(X_future_t, y_future_t)

In [12]:
# Hiperparámetros de PyTorch
hidden_dims = [64, 128]
learning_rates = [0.001, 0.002, 0.003]
dropouts = [0.2, 0.3]
epochs_list = [40, 50, 60, 70, 80, 90]
batch_size = 128  # Fijo para este ejemplo

combinaciones = list(itertools.product(hidden_dims, learning_rates, dropouts, epochs_list))
print(f"Probando {len(combinaciones)} combinaciones de hiperparámetros de PyTorch...")

data_test = []
data_2025 = []
modelos = []
iteracion = 0
for hidden, lr, drop, epochs in tqdm(combinaciones, desc="Entrenando modelos PyTorch"):
    # Crear DataLoaders para la iteración actual
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    future_loader = DataLoader(future_dataset, batch_size=batch_size, shuffle=False)
    
    datos_dict_test, datos_dict_real, modelo = entrena_evalua_pytorch(
        hidden_dim=hidden, 
        learning_rate=lr, 
        dropout=drop, 
        epochs=epochs,
        batch_size=batch_size, 
        train_loader=train_loader, 
        test_loader=test_loader,
        future_loader=future_loader, 
        df_test=df_test, 
        df_future=df_future, 
        device=device,
        iteracion=iteracion
    )
    data_test.append(datos_dict_test)
    data_2025.append(datos_dict_real)
    modelos.append(modelo)
    
    iteracion += 1

df_parametros_data_test = pl.DataFrame(data_test)
df_parametros_data_2025 = pl.DataFrame(data_2025)

Probando 72 combinaciones de hiperparámetros de PyTorch...


Entrenando modelos PyTorch:   0%|          | 0/72 [00:00<?, ?it/s]

In [ ]:
'''
directional_accuracy   0.564103   con [price, total_volume, stock_index_sp500, price_gold, rate_US10Y, stock_index_ni225, stock_index_dowjones]

directional_accuracy   0.582418   con [price, total_volume, stock_index_sp500, price_gold,  rate_US10Y, stock_index_ni225]

directional_accuracy   0.545788   con [price, total_volume, stock_index_sp500, stock_index_ni225]

directional_accuracy   0.534799   con [price, total_volume, stock_index_sp500, price_gold]

directional_accuracy   0.538462   con [price, total_volume, stock_index_sp500, rate_US10Y]

directional_accuracy   0.567766   con [price, total_volume, stock_index_sp500]

directional_accuracy   0.555147   con [price, total_volume]

directional_accuracy   0.540441   con [price]

'''

'\ndirectional_accuracy    ########   y   0.564103   con [price, total_volume, stock_index_sp500, price_gold, rate_US10Y, stock_index_ni225, stock_index_dowjones]\n\ndirectional_accuracy    ########   y   0.582418   con [price, total_volume, stock_index_sp500, price_gold,  rate_US10Y, stock_index_ni225]\n\ndirectional_accuracy    ########   y   0.545788   con [price, total_volume, stock_index_sp500, stock_index_ni225]\n\ndirectional_accuracy    ########   y   0.534799   con [price, total_volume, stock_index_sp500, price_gold]\n\ndirectional_accuracy    ########   y   0.538462   con [price, total_volume, stock_index_sp500, rate_US10Y]\n\ndirectional_accuracy    ########   y   0.567766   con [price, total_volume, stock_index_sp500]\n\ndirectional_accuracy    ########   y   0.555147   con [price, total_volume]\n\ndirectional_accuracy    ########   y   0.540441   con [price]\n\n'

#### Estos son los parámetros del Modelo con Mejor `directional_acrruracy`

In [14]:
df_parametros_data_2025.sort("directional_accuracy", descending=True).head(1)

directional_accuracy,sharpe_ratio,cumulative_return,Compound_Annual_Growth_Rate,n_evaluated_days,input_dim,hidden_dims,learning_rate,dropout,epochs,batch_size,iteracion
f64,f64,f64,f64,i64,i64,list[i64],f64,f64,i64,i64,i64
0.582418,0.077726,0.44833,0.64089,273,42,"[64, 32, 16]",0.003,0.2,60,128,26


### Recuperando el Modelo y sus Características del Modelo con mejor `directional_accuracy`

In [15]:
# Encontrar el mejor modelo
parametros_mejor_modelo = df_parametros_data_2025.sort("directional_accuracy", descending=True).head(1).to_dicts()[0]

modelo_comprobacion = modelos[parametros_mejor_modelo['iteracion']]
instancia_dict = {
    'input_dim': parametros_mejor_modelo['input_dim'],
    'hidden_dims': parametros_mejor_modelo['hidden_dims'],
    'dropout_prob': parametros_mejor_modelo['dropout'],
}


#### Comprobación

In [16]:
import copy

def comprobacion(model, scaler):

    # Columnas Features
    features_pytorch = [col for col in df_future.columns if col not in ["date", 'target_direction', 'price_tomorrow']]

    # Modelo tal cual sera utilizado en Produccion
    pytorch_model = model
    pytorch_model.eval()

    # device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Preparacion del X_input para PyTorch 
    
    # X_input_pytorch = scaler.transform(df_future.sort('date').select(features_pytorch).to_numpy())
    X_input_pytorch = X_test_future
    X_input_pytorch_t = torch.tensor(X_input_pytorch, dtype=torch.float32)
    # Inferencia 
    with torch.no_grad():
        y_pred_pytorch_t = pytorch_model(X_input_pytorch_t.to(device))

    preds = y_pred_pytorch_t.cpu().detach().numpy().flatten()  
    y_test = copy.deepcopy(y_test_future)

    aciertos_direccion = 0
    for i in range(len(preds)):
        if (preds[i] > 0.5 and y_test[i] == 1) or (preds[i] < 0.5 and y_test[i] == -1):
            aciertos_direccion += 1
  
    directional_accuracy = aciertos_direccion / len(preds)
    return directional_accuracy


directional_data_2025 = comprobacion(modelo_comprobacion, scaler_global)
print(f"directional_accuracy --> {directional_data_2025}")

directional_accuracy --> 0.5824175824175825


***
***
### 💾 Guardando los Modelos

In [ ]:
import json
import joblib 

torch.save(modelo_comprobacion.state_dict(), 'models/pytorch_model_data_2025_binario.pth')

with open("models/config_pytorch_2025_binario.json", "w") as f:
    json.dump(instancia_dict, f, indent=4)

scaler_filename = "models/scaler_global_pytorch_binario.joblib"
joblib.dump(scaler_global, scaler_filename)   